In [ ]:
#datasets path

#AD100

TRAIN_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/train.csv"
TEST_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/verification-subject-disjoint.csv"
IMAGE_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/images/"

#LG4000

TRAIN_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/train.csv"
TEST_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/verification-subject-disjoint.csv"
IMAGE_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/images/"

In [ ]:
#Classifier Functions

def contacts_classifier(label):
  if label == "No":
    return [0, 0, 1]
  elif label == "Yes":
    return [0, 1, 0]
  elif label == "Cosmetic":
    return [1, 0, 0]



def eye_classifier(label):
  if label == "Right":
    return [0, 1]
  elif label == "Left":
    return [1, 0]

In [ ]:
import csv
import numpy as np

#Reading the data


def read_csv(filepath):
  lines = []

  with open(filepath, "r") as csv_read:
    csvreader = csv.reader(csv_read)

    for row in csvreader:
      lines.append(np.asarray(row))
    csv_read.close()

    lines.pop(0)
    lines = np.array(lines).T

    return lines

lines = read_csv(TRAIN_PATH_LG)
print(lines)

[['04261d1016' '04261d1033' '04261d1036' ... '07015d92' '07015d93'
  '07015d99']
 ['nd1S04261' 'nd1S04261' 'nd1S04261' ... 'nd1S07015' 'nd1S07015'
  'nd1S07015']
 ['Left' 'Right' 'Right' ... 'Right' 'Right' 'Right']
 ...
 ['234' '246' '233' ... '234' '235' '236']
 ['124' '122' '132' ... '131' '128' '132']
 ['6' '3' '3' ... '0' '0' '0']]


In [ ]:
def convert_classes(filepath):
  csv_data = read_csv(filepath)

  eye, contacts = [], []

  for label1 in csv_data[5]:
    contacts.append(contacts_classifier(label1))

  for label2 in csv_data[2]:
    eye.append(eye_classifier(label2))

  return np.array(eye).astype(np.int32), np.array(contacts).astype(np.int32)

In [ ]:
y_train_eye, y_train_contact = convert_classes(TRAIN_PATH_LG)
y_test_eye, y_test_contact = convert_classes(TEST_PATH_LG)

print(y_train_eye.shape, y_train_contact.shape)
print(y_test_eye.shape, y_test_contact.shape)

(3000, 2) (3000, 3)
(1200, 2) (1200, 3)


In [ ]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import tensorflow as tf

def read_images(filepath, augment=False):
  csv_data = read_csv(filepath)
  images_names = csv_data[0]

  images = []

  for item in images_names:
    img_path = IMAGE_PATH_LG + item + ".tiff"
    image = img_to_array(load_img(img_path, target_size=(299, 299))) / 255.0

    images.append(image)

  return np.array(images).astype(np.float32)

In [ ]:
x_train = read_images(TRAIN_PATH_LG)
x_test = read_images(TEST_PATH_LG)

print(x_train.shape)
print(x_test.shape)

(3000, 299, 299, 3)
(1200, 299, 299, 3)


In [ ]:
from keras.applications import Xception

base_model = Xception(include_top=False, weights='imagenet')
base_model.trainable = False

base_model.summary()

83683744/83683744 [==============================] - 0s 0us/step
Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, None, None, 32)       128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activat

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Flatten
from keras.applications import xception


input_layer = Input(shape=(299, 299, 3))

input_layer = xception.preprocess_input(input_layer)

base_model = base_model (input_layer)

batch_norm = BatchNormalization() (base_model)
global_pooling = GlobalAveragePooling2D() (batch_norm)

dropout1 = Dropout(0.2)(global_pooling)
dense1 = Dense(1024, activation='relu')(dropout1)

dropout2 = Dropout(0.2)(dense1)
dense2 = Dense(512, activation='relu')(dropout2)

dropout3 = Dropout(0.2)(dense2)
dense3 = Dense(512, activation='relu')(dropout3)

final_dropout = Dropout(0.1)(dense3)
final_dense   = Dense(32, activation='relu')(final_dropout)

#contacts layer
contacts_layer = Dense(3, activation='softmax', name="contacts_layer")(final_dense)

#eye layer
eye_layer = Dense(2, activation='sigmoid', name="eye_layer")(final_dense)

model = Model(inputs= input_layer, outputs=[eye_layer, contacts_layer])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 xception (Functional)       (None, None, None, 2048)     2086148   ['input_3[0][0]']             
                                                          0                                       
                                                                                                  
 batch_normalization_4 (Bat  (None, 10, 10, 2048)         8192      ['xception[1][0]']            
 chNormalization)                                                                                 
                                                                                              

In [ ]:
from keras.optimizers import RMSprop, Adam, SGD
from keras.optimizers.schedules import ExponentialDecay
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy

learning_rates = [0.004, 0.0001]
epochs = 20
batch_size = 5
decay_factor = (learning_rates[1] / learning_rates[0]) / epochs
decay_steps = int(len(x_train) / batch_size)

learning_rate = ExponentialDecay(
    initial_learning_rate= learning_rates[0],
    decay_steps= decay_steps,
    decay_rate=decay_factor,
    staircase=True
)

optimizer = Adam(learning_rate=learning_rate)


losses = {
    'eye_layer': BinaryCrossentropy(),
    'contacts_layer': CategoricalCrossentropy()
}

model.compile(optimizer= optimizer, loss= losses, metrics= 'accuracy')

In [ ]:
history = model.fit(x_train, [y_train_eye, y_train_contact], epochs= epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)

Epoch 1/20
480/480 [==============================] - 38s 55ms/step - loss: 1.7653 - eye_layer_loss: 0.7064 - contacts_layer_loss: 1.0589 - eye_layer_accuracy: 0.5104 - contacts_layer_accuracy: 0.4087 - val_loss: 2.3769 - val_eye_layer_loss: 0.6925 - val_contacts_layer_loss: 1.6844 - val_eye_layer_accuracy: 0.5267 - val_contacts_layer_accuracy: 0.0000e+00
Epoch 2/20
480/480 [==============================] - 25s 51ms/step - loss: 1.7215 - eye_layer_loss: 0.6929 - contacts_layer_loss: 1.0286 - eye_layer_accuracy: 0.5142 - contacts_layer_accuracy: 0.4117 - val_loss: 2.4497 - val_eye_layer_loss: 0.6923 - val_contacts_layer_loss: 1.7574 - val_eye_layer_accuracy: 0.5267 - val_contacts_layer_accuracy: 0.0000e+00
Epoch 3/20
480/480 [==============================] - 25s 52ms/step - loss: 1.7211 - eye_layer_loss: 0.6928 - contacts_layer_loss: 1.0283 - eye_layer_accuracy: 0.5142 - contacts_layer_accuracy: 0.4167 - val_loss: 2.4498 - val_eye_layer_loss: 0.6923 - val_contacts_layer_loss: 1.7575 -

In [ ]:
#fine-tuning

base_model.trainable = True
model.summary(show_trainable=True)

Model: "model"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            Y          
                                                                                                             
 xception (Functional)       (None, None, None, 2048)     2086148   ['input_3[0][0]']             N          
                                                          0                                                  
                                                                                                             
 batch_normalization_4 (Bat  (None, 10, 10, 2048)         8192      ['xception[1][0]']            Y          
 chNormalization)                                                                                        

In [ ]:
end_optimizer = optimizer = Adam(learning_rate=1e-5)


model.compile(optimizer= end_optimizer, loss= losses, metrics= 'accuracy')

In [ ]:
final_epochs = 10


end_history = model.fit(x_train, [y_train_eye, y_train_contact], epochs= final_epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)

In [ ]:
evaluation = model.evaluate(x_test, [y_test_eye, y_test_contact], verbose=1)

In [ ]:
print("Eye's Prediction:     ",evaluation[3] * 100,"%")
print("Contact's Prediction: ",evaluation[4] * 100,"%")